# 21522557 - Trần Thanh Sơn
# 21522678 - Phạm Trung Tín

In [ ]:
# TEAM 12

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install pyspark findspark

In [ ]:
import pandas as pd
import findspark
findspark.init()
import pyspark
from pyspark.sql.functions import when, col
from pyspark.sql.functions import expr
from pyspark.ml import PipelineModel

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").getOrCreate()

In [ ]:
test_df = spark.read.csv("/kaggle/input/uit-diabetic-patients-re-admission-prediction/test_diabetic_data.csv", header=True, inferSchema=True)

def count_question_marks_per_column(df):

    counts = {}

    for column in df.columns:
        count_question_mark = df.filter(col(column) == "?").count()
        counts[column] = count_question_mark

    return counts
question_mark_counts = count_question_marks_per_column(test_df)
for column, count in question_mark_counts.items():
    if count >0:
      print(f"Số lượng giá trị '?' trong cột '{column}': {count}")
test_df = test_df.withColumn("weight", when(col("weight") == "?", None).otherwise(col("weight")))
test_df = test_df.withColumn("race", when(col("race") == "?", None).otherwise(col("race")))
test_df = test_df.withColumn("payer_code", when(col("payer_code") == "?", None).otherwise(col("payer_code")))
test_df = test_df.withColumn("medical_specialty", when(col("medical_specialty") == "?", None).otherwise(col("medical_specialty")))
test_df = test_df.withColumn("diag_1", when(col("diag_1") == "?", None).otherwise(col("diag_1")))
test_df = test_df.withColumn("diag_2", when(col("diag_2") == "?", None).otherwise(col("diag_2")))
test_df = test_df.withColumn("diag_3", when(col("diag_3") == "?", None).otherwise(col("diag_3")))

imp_col = ['race','gender','age','admission_type_id','discharge_disposition_id',
           'admission_source_id','time_in_hospital','num_lab_procedures','num_procedures',
           'num_medications','number_outpatient','number_emergency','number_inpatient',
           'diag_1','diag_2','diag_3','number_diagnoses','max_glu_serum','A1Cresult','insulin','change', 'diabetesMed']

test_df = test_df.select(imp_col)
from pyspark.ml.feature import StringIndexer

string_cols = ['race', 'gender', 'age',
               'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'insulin', 'change', 'diabetesMed']
df_transformed = test_df  # Initialize with the original DataFrame

string_indexer = PipelineModel.load("/kaggle/input/mnodel/string_indexer_only_model")

df_transformed = string_indexer.transform(df_transformed)

columns_to_drop = ['race', 'gender', 'age',
                   'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'insulin', 'change', 'diabetesMed']

df_transformed = df_transformed.drop(*columns_to_drop)

# Initialize the DataFrame to the original DataFrame
final_df = df_transformed


# Perform groupBy operation on the final DataFrame
for column in final_df.columns:
    column_counts = final_df.groupBy(column).count()
    print(f"Counts for Column '{column}':")
    column_counts.show(30)


final_df = final_df.withColumn('race_new', when((col('race_new') == 5.0), None).otherwise(col('race_new')))
final_df = final_df.withColumn('diag_1_new', when((col('diag_1_new') == 559.0), None).otherwise(col('diag_1_new')))
final_df = final_df.withColumn('diag_2_new', when((col('diag_2_new') == 547.0), None).otherwise(col('diag_2_new')))
final_df = final_df.withColumn('diag_3_new', when((col('diag_3_new') == 605.0), None).otherwise(col('diag_3_new')))



In [ ]:
from pyspark.ml.feature import ImputerModel
# Điền các giá trị còn thiếu
imputer_model = ImputerModel.load("/kaggle/input/mnodel/imputer_model")
test_data= imputer_model.transform(final_df)

In [ ]:


df_processed = test_data.withColumn('race_new', expr('CAST(ROUND(race_new) AS INT)'))
test_data = df_processed 
df_processed.show()

In [ ]:
modeld = PipelineModel.load("/kaggle/input/mnodel/pinelinexgb_model")

prediction = modeld.transform(test_data)
count = prediction.groupBy("prediction").count()
count.show(30)

In [ ]:
prediction.count()

In [ ]:
prediction = prediction.select("prediction")
pred = prediction.toPandas()
pred

In [ ]:
Id = spark.read.csv("/kaggle/input/uit-diabetic-patients-re-admission-prediction/test_diabetic_data.csv", header=True, inferSchema=True).toPandas()[['index']]
Id

In [ ]:
# Gộp DataFrame theo cột thứ tự dòng
merged_df = pd.merge(Id, pred, how='outer',left_index=True, right_index=True)
merged_df

In [ ]:
submit_df = spark.createDataFrame(merged_df.values.tolist(), merged_df.columns.tolist())
submit_df = submit_df.withColumn("index", submit_df["index"].cast("int"))
submit_df = submit_df.withColumn("prediction", submit_df["prediction"].cast("int"))
submit_df.show(100)

In [ ]:
submit_df = submit_df.select("index", "prediction")
submit_df = submit_df.withColumn(
    "prediction",
    when(submit_df["prediction"] == 0, "NO")
    .when(submit_df["prediction"] == 1, "GT30")
    .otherwise("LT30")
)
submit_df = submit_df.withColumnRenamed("prediction", "readmitted")
submit_df.show(10)

In [ ]:
submit_df.groupBy("readmitted").count().show()

In [ ]:
submit_df.toPandas()


In [ ]:
# Ghi DataFrame vào tệp CSV
submit_df.toPandas().to_csv("out.csv", index = False)